Git clone the repo and install the requirements. (ignore the pip errors about protobuf)

In [1]:
# #@title Environment Setup

from pathlib import Path
import os

OPTIONS = {}

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
USE_COMFYUI_MANAGER = True  #@param {type:"boolean"}
INSTALL_CUSTOM_NODES_DEPENDENCIES = True  #@param {type:"boolean"}
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI
OPTIONS['USE_COMFYUI_MANAGER'] = USE_COMFYUI_MANAGER
OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES'] = INSTALL_CUSTOM_NODES_DEPENDENCIES

current_dir = !pwd
WORKSPACE = f"{current_dir[0]}/ComfyUI"

# 🔹 FIX: definir COMFYUI_PATH para evitar warning
os.environ["COMFYUI_PATH"] = WORKSPACE

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    os.environ["COMFYUI_PATH"] = WORKSPACE   # actualizar también aquí

    %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-

  # Correction of the issue of permissions being deleted on Google Drive.
  ![ -f ".ci/nightly/update_windows/update_comfyui_and_python_dependencies.bat" ] && chmod 755 .ci/nightly/update_windows/update_comfyui_and_python_dependencies.bat
  ![ -f ".ci/nightly/windows_base_files/run_nvidia_gpu.bat" ] && chmod 755 .ci/nightly/windows_base_files/run_nvidia_gpu.bat
  ![ -f ".ci/update_windows/update_comfyui_and_python_dependencies.bat" ] && chmod 755 .ci/update_windows/update_comfyui_and_python_dependencies.bat
  ![ -f ".ci/update_windows_cu118/update_comfyui_and_python_dependencies.bat" ] && chmod 755 .ci/update_windows_cu118/update_comfyui_and_python_dependencies.bat
  ![ -f ".ci/update_windows/update.py" ] && chmod 755 .ci/update_windows/update.py
  ![ -f ".ci/update_windows/update_comfyui.bat" ] && chmod 755 .ci/update_windows/update_comfyui.bat
  ![ -f ".ci/update_windows/README_VERY_IMPORTANT.txt" ] && chmod 755 .ci/update_windows/README_VERY_IMPORTANT.txt
  ![ -f ".ci/update_windows/run_cpu.bat" ] && chmod 755 .ci/update_windows/run_cpu.bat
  ![ -f ".ci/update_windows/run_nvidia_gpu.bat" ] && chmod 755 .ci/update_windows/run_nvidia_gpu.bat

  !git pull

!echo -= Install dependencies =-
!pip3 install accelerate
!pip3 install einops transformers>=4.28.1 safetensors>=0.4.2 aiohttp pyyaml Pillow scipy tqdm psutil tokenizers>=0.13.3
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip3 install torchsde
!pip3 install kornia>=0.7.1 spandrel soundfile sentencepiece

if OPTIONS['USE_COMFYUI_MANAGER']:
  %cd custom_nodes

  # Correction of the issue of permissions being deleted on Google Drive.
  ![ -f "ComfyUI-Manager/check.sh" ] && chmod 755 ComfyUI-Manager/check.sh
  ![ -f "ComfyUI-Manager/scan.sh" ] && chmod 755 ComfyUI-Manager/scan.sh
  ![ -f "ComfyUI-Manager/node_db/dev/scan.sh" ] && chmod 755 ComfyUI-Manager/node_db/dev/scan.sh
  ![ -f "ComfyUI-Manager/node_db/tutorial/scan.sh" ] && chmod 755 ComfyUI-Manager/node_db/tutorial/scan.sh
  ![ -f "ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh" ] && chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh
  ![ -f "ComfyUI-Manager/scripts/install-comfyui-venv-win.bat" ] && chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-win.bat

  ![ ! -d ComfyUI-Manager ] && echo -= Initial setup ComfyUI-Manager =- && git clone https://github.com/ltdrdata/ComfyUI-Manager
  %cd ComfyUI-Manager
  !git pull

%cd $WORKSPACE

if OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES']:
  !echo -= Install custom nodes dependencies =-
  !pip install GitPython
  !python custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies


Mounting Google Drive...
/
Mounted at /content/drive
/content/drive/MyDrive
-= Initial setup ComfyUI =-
Cloning into 'ComfyUI'...
remote: Enumerating objects: 23222, done.
remote: Counting objects: 100% (224/224), done.
remote: Compressing objects: 100% (132/132), done.
remote: Total 23222 (delta 179), reused 92 (delta 92), pack-reused 22998 (from 3)
Receiving objects: 100% (23222/23222), 72.44 MiB | 10.25 MiB/s, done.
Resolving deltas: 100% (15534/15534), done.
Updating files: 100% (501/501), done.
/content/drive/MyDrive/ComfyUI
-= Updating ComfyUI =-
Already up to date.
-= Install dependencies =-
Looking in indexes: https://download.pytorch.org/whl/cu121
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 2.2 MB/s eta 0:00:00
/content/drive/MyDrive/ComfyUI/custom_nodes
-= Initial setup ComfyUI-Manager =-
Cloning into 'ComfyUI-Manager'...
remote: Enumerating objects: 23508, done.
remote: Counting objects: 100% (368/368), done.
remote: Compressing objects: 100% (207/207), done.
re

Download some models/checkpoints/vae or custom comfyui nodes (uncomment the commands for the ones you want)

In [ ]:
# #@title DESCARGAR: Workflows (Ejemplos) + Modelos/Pesos (robusto, con HF_TOKEN opcional)
import os, subprocess, glob, html
from pathlib import Path

# === Paths base (usa lo definido en Environment Setup) ===
WORKSPACE = os.environ.get("COMFYUI_PATH", "/content/drive/MyDrive/ComfyUI")
MODELS    = Path(WORKSPACE) / "models"
EX_ROOT   = Path(WORKSPACE) / "examples"
EX_DIR    = EX_ROOT / "ComfyUI_examples"
EX_ROOT.mkdir(parents=True, exist_ok=True)
MODELS.mkdir(parents=True, exist_ok=True)

# === 0) Asegurar subcarpetas de modelos ===
for d in ["checkpoints","vae","loras","controlnet","clip_vision","style_models","gligen","upscale_models"]:
    (MODELS / d).mkdir(parents=True, exist_ok=True)

# === 1) Instalar aria2c si no está (descargas rápidas/robustas) ===
def ensure_aria2():
    try:
        subprocess.run(["aria2c","-v"], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        return True
    except Exception:
        pass
    try:
        subprocess.run(["apt-get","update","-y"], check=True)
        subprocess.run(["apt-get","install","-y","aria2"], check=True)
        subprocess.run(["aria2c","-v"], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        return True
    except Exception:
        return False

HAS_ARIA2 = ensure_aria2()
print("aria2c:", "OK" if HAS_ARIA2 else "NO")

# === 2) Token de HF opcional desde Colab Secrets o env ===
HF_TOKEN = None
try:
    from google.colab import userdata
    HF_TOKEN = userdata.get('HF_TOKEN')
except Exception:
    HF_TOKEN = os.environ.get("HF_TOKEN", None)

# === 3) Función de descarga (aria2c -> wget fallback), con Bearer si es HF ===
def dl(url: str, out_dir: Path, out_name: str|None=None):
    out_dir.mkdir(parents=True, exist_ok=True)
    headers = []
    if "huggingface.co" in url and HF_TOKEN:
        headers.append("Authorization: Bearer " + HF_TOKEN)

    if HAS_ARIA2:
        cmd = ["aria2c","-c","-x16","-s16","-m","20","--retry-wait=5","--console-log-level=warn", url, "-d", str(out_dir)]
        if out_name:
            cmd += ["-o", out_name]
        for h in headers:
            cmd += ["--header", h]
    else:
        # wget no mezcla bien -P con -O; si renombramos usamos -O
        if out_name:
            cmd = ["wget","-c","-t","20","--timeout=30", url, "-O", str(out_dir/out_name)]
        else:
            cmd = ["wget","-c","-t","20","--timeout=30", url, "-P", str(out_dir)]
        for h in headers:
            cmd += ["--header", h]

    print("->", url)
    if headers: print("   + Auth: Bearer HF_TOKEN")
    subprocess.run(cmd, check=False)

# === 4) CLONAR/ACTUALIZAR WORKFLOWS (ComfyUI_examples) + índice HTML ===
if not EX_DIR.exists():
    print("Clonando ComfyUI_examples ...")
    subprocess.run(["git","clone","--depth=1","https://github.com/comfyanonymous/ComfyUI_examples.git", str(EX_DIR)], check=True)
else:
    print("Actualizando ComfyUI_examples ...")
    subprocess.run(["bash","-lc", f"cd '{EX_DIR}' && git pull --rebase --autostash"], check=False)

pngs = sorted(glob.glob(str(EX_DIR / "**/*.png"), recursive=True))
index_html = EX_ROOT / "index.html"
cards = []
for p in pngs:
    rel = os.path.relpath(p, EX_ROOT)
    cards.append(
        f'<div style="display:inline-block;margin:6px;text-align:center;">'
        f'<a href="{html.escape(rel)}" target="_blank">'
        f'<img src="{html.escape(rel)}" style="width:240px;border:1px solid #444;border-radius:6px;display:block"/></a>'
        f'<small style="color:#ccc;">{html.escape(os.path.relpath(p, EX_DIR))}</small></div>'
    )
index_html.write_text(f"""<!doctype html>
<html><head><meta charset="utf-8"><title>ComfyUI Examples (local)</title></head>
<body style="background:#111;color:#eee;font-family:system-ui,Segoe UI,Roboto,Arial;">
<h2>ComfyUI Examples (local)</h2>
<p>Arrastra un PNG a la ventana de ComfyUI o usa el botón <b>Load</b> para cargar el workflow desde sus metadatos.</p>
<div>{"".join(cards)}</div>
</body></html>""", encoding="utf-8")
print(f"Workflows listos: {len(pngs)} PNGs")
print("Carpeta:", EX_DIR)
print("Índice HTML:", index_html)

# === 5) LISTA DE MODELOS/PESOS A DESCARGAR ===
items = [
    # SDXL
    ("https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors", MODELS/"checkpoints", None),
    ("https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors", MODELS/"checkpoints", None),

    # SDXL ReVision (CLIP Vision G)
    ("https://huggingface.co/comfyanonymous/clip_vision_g/resolve/main/clip_vision_g.safetensors", MODELS/"clip_vision", None),

    # SD1.5
    ("https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt", MODELS/"checkpoints", None),

    # SD2.x base y 768
    ("https://huggingface.co/stabilityai/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.safetensors", MODELS/"checkpoints", None),
    ("https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.safetensors", MODELS/"checkpoints", None),

    # Anime SD1.5
    ("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix2/AbyssOrangeMix2_hard.safetensors", MODELS/"checkpoints", None),
    ("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A1_orangemixs.safetensors", MODELS/"checkpoints", None),
    ("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A3_orangemixs.safetensors", MODELS/"checkpoints", None),
    ("https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/anything-v3-fp16-pruned.safetensors", MODELS/"checkpoints", None),

    # Waifu Diffusion 1.5 (anime SD2.x 768-v)
    ("https://huggingface.co/waifu-diffusion/wd-1-5-beta3/resolve/main/wd-illusion-fp16.safetensors", MODELS/"checkpoints", None),

    # unCLIP
    ("https://huggingface.co/comfyanonymous/illuminatiDiffusionV1_v11_unCLIP/resolve/main/illuminatiDiffusionV1_v11-unclip-h-fp16.safetensors", MODELS/"checkpoints", None),
    ("https://huggingface.co/comfyanonymous/wd-1.5-beta2_unCLIP/resolve/main/wd-1-5-beta2-aesthetic-unclip-h-fp16.safetensors", MODELS/"checkpoints", None),

    # VAEs
    ("https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors", MODELS/"vae", None),
    ("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt", MODELS/"vae", None),
    ("https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt", MODELS/"vae", None),

    # LoRAs
    ("https://civitai.com/api/download/models/10350", MODELS/"loras", "theovercomer8sContrastFix_sd21768.safetensors"),
    ("https://civitai.com/api/download/models/10638", MODELS/"loras", "theovercomer8sContrastFix_sd15.safetensors"),
    ("https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_offset_example-lora_1.0.safetensors", MODELS/"loras", None),

    # T2I-Adapter (ControlNet-like)
    ("https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_depth_sd14v1.pth", MODELS/"controlnet", None),
    ("https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_seg_sd14v1.pth", MODELS/"controlnet", None),
    ("https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_sketch_sd14v1.pth", MODELS/"controlnet", None),
    ("https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_keypose_sd14v1.pth", MODELS/"controlnet", None),
    ("https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_openpose_sd14v1.pth", MODELS/"controlnet", None),
    ("https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth", MODELS/"controlnet", None),
    ("https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_canny_sd14v1.pth", MODELS/"controlnet", None),
    ("https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth", MODELS/"style_models", None),

    # CLIP Vision L/14 (para styles)
    ("https://huggingface.co/openai/clip-vit-large-patch14/resolve/main/pytorch_model.bin", MODELS/"clip_vision", "clip_vit14.bin"),

    # ControlNet 1.1 (SD1.5)
    ("https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors", MODELS/"controlnet", None),
    ("https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors", MODELS/"controlnet", None),
    ("https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_canny_fp16.safetensors", MODELS/"controlnet", None),
    ("https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors", MODELS/"controlnet", None),
    ("https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors", MODELS/"controlnet", None),
    ("https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_lineart_fp16.safetensors", MODELS/"controlnet", None),
    ("https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors", MODELS/"controlnet", None),
    ("https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors", MODELS/"controlnet", None),
    ("https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_openpose_fp16.safetensors", MODELS/"controlnet", None),
    ("https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_scribble_fp16.safetensors", MODELS/"controlnet", None),
    ("https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_seg_fp16.safetensors", MODELS/"controlnet", None),
    ("https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_softedge_fp16.safetensors", MODELS/"controlnet", None),
    ("https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors", MODELS/"controlnet", None),
    ("https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11u_sd15_tile_fp16.safetensors", MODELS/"controlnet", None),

    # ControlNet SDXL (LoRA rank256)
    ("https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-canny-rank256.safetensors", MODELS/"controlnet", None),
    ("https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-depth-rank256.safetensors", MODELS/"controlnet", None),
    ("https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-recolor-rank256.safetensors", MODELS/"controlnet", None),
    ("https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-sketch-rank256.safetensors", MODELS/"controlnet", None),

    # GLIGEN
    ("https://huggingface.co/comfyanonymous/GLIGEN_pruned_safetensors/resolve/main/gligen_sd14_textbox_pruned_fp16.safetensors", MODELS/"gligen", None),

    # ESRGAN
    ("https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth", MODELS/"upscale_models", None),
    ("https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x2.pth", MODELS/"upscale_models", None),
    ("https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x4.pth", MODELS/"upscale_models", None),
]

print("\nDescargando MODELOS/PESOS...")
for url, out_dir, rename in items:
    dl(url, out_dir, rename)

# === 6) Preprocessors para ControlNet (Fannovel16) ===
cn_dir = Path(WORKSPACE) / "custom_nodes" / "comfy_controlnet_preprocessors"
if not cn_dir.exists():
    try:
        print("\nClonando comfy_controlnet_preprocessors ...")
        subprocess.run(["bash","-lc", f"cd '{Path(WORKSPACE)/'custom_nodes'}' && git clone https://github.com/Fannovel16/comfy_controlnet_preprocessors"], check=True)
        print("Instalando preprocessors ...")
        subprocess.run(["bash","-lc", f"cd '{cn_dir}' && python install.py"], check=True)
    except Exception as e:
        print("WARN: Falló la instalación de comfy_controlnet_preprocessors:", e)

print("\n✅ Todo listo.")
print("Workflows:", EX_DIR)
print("Índice:", index_html)
print("Modelos:", MODELS)
print("Usa ComfyUI: arrastra un PNG de 'examples/ComfyUI_examples' o Load → PNG.")


### Run ComfyUI with cloudflared (Recommended Way)




In [ ]:
!wget -P ~ https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i ~/cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

### Run ComfyUI with localtunnel




In [ ]:
!npm install -g localtunnel

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")

  print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
  p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
  for line in p.stdout:
    print(line.decode(), end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

### Run ComfyUI with colab iframe (use only in case the previous way with localtunnel doesn't work)

You should see the ui appear in an iframe. If you get a 403 error, it's your firefox settings or an extension that's messing things up.

If you want to open it in another window use the link.

Note that some UI features like live image previews won't work because the colab iframe blocks websockets.

In [ ]:
import threading
import time
import socket
def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  from google.colab import output
  output.serve_kernel_port_as_iframe(port, height=1024)
  print("to open it in a window you can open this link here:")
  output.serve_kernel_port_as_window(port)

threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server